In [2]:
# Self-contained cell to load and join both datasets
from datasets import load_dataset
import pandas as pd

# Load both datasets
print("Loading datasets...")
ds_qs = load_dataset("launch/gov_report_qs", "document")
ds_gr = load_dataset("launch/gov_report", trust_remote_code=True)

# Convert gov_report_qs splits to pandas
train_qs = ds_qs["train"].to_pandas()
val_qs = ds_qs["validation"].to_pandas()
test_qs = ds_qs["test"].to_pandas()

# Convert gov_report splits to pandas and combine ALL splits
train_gr = ds_gr["train"].to_pandas()
val_gr = ds_gr["validation"].to_pandas()
test_gr = ds_gr["test"].to_pandas()
all_gr = pd.concat([train_gr, val_gr, test_gr], ignore_index=True)

print(f"\nDataset sizes:")
print(f"gov_report_qs - train: {len(train_qs)}, val: {len(val_qs)}, test: {len(test_qs)}")
print(f"gov_report - total: {len(all_gr)}")

# Join each split of gov_report_qs against the full gov_report dataset
train_combined = train_qs.merge(all_gr, left_on='doc_id', right_on='id', how='inner')
val_combined = val_qs.merge(all_gr, left_on='doc_id', right_on='id', how='inner')
test_combined = test_qs.merge(all_gr, left_on='doc_id', right_on='id', how='inner')

print(f"\nCombined dataset sizes:")
print(f"  train: {len(train_combined)} (kept {len(train_combined)/len(train_qs)*100:.1f}%)")
print(f"  val: {len(val_combined)} (kept {len(val_combined)/len(val_qs)*100:.1f}%)")
print(f"  test: {len(test_combined)} (kept {len(test_combined)/len(test_qs)*100:.1f}%)")
print(f"  total: {len(train_combined) + len(val_combined) + len(test_combined)}")

print(f"\nColumns: {list(train_combined.columns)}")
print("\nTrain split preview:")
train_combined.head()

/home/ubuntu/reflective-equilibrium/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading datasets...

Dataset sizes:
gov_report_qs - train: 1371, val: 171, test: 172
gov_report - total: 19466

Combined dataset sizes:
  train: 1371 (kept 100.0%)
  val: 171 (kept 100.0%)
  test: 172 (kept 100.0%)
  total: 1714

Columns: ['doc_id', 'document_sections', 'question_summary_pairs', 'id', 'document', 'summary']

Train split preview:


,doc_id,document_sections,question_summary_pairs,id,document,summary
0,GAO_GAO-12-604,"{'title': ['Background', 'Polar Satellite Data...",{'question': ['What did NOAA and DOD do follow...,GAO_GAO-12-604,"Background Since the 1960s, the United States ...",Environmental satellites provide critical data...
1,GAO_GAO-13-174,"{'title': ['Background', 'Agencies Have Made P...",{'question': ['What did GAO's review of APGs f...,GAO_GAO-13-174,Background GPRAMA specifies that APGs are to h...,GAOs work has repeatedly shown that federal a...
2,GAO_GAO-12-843,"{'title': ['Background', 'Six Agencies Support...",{'question': ['What solar initiatives did fede...,GAO_GAO-12-843,"Background Solar energy can be used to heat, c...",The United States has abundant solar energy re...
3,CRS_R42983,"{'title': ['', 'Background', 'Rationale for th...",{'question': ['For what are scientific studies...,CRS_R42983,The results of scientific studies are used in ...,The results of scientific studies are often us...
4,GAO_GAO-15-236,"{'title': ['Background', 'FHWA Has Established...",{'question': ['What regulations has the FHWA e...,GAO_GAO-15-236,Background Commercial vehicles and loads are g...,"In May 2013, a truck carrying an oversize load..."


In [3]:
# Check if summary_paragraph_index exists
row_index = 0
qa_pairs = train_combined['question_summary_pairs'][row_index]

print("Fields in question_summary_pairs:")
print(qa_pairs.keys())
print()

if 'summary_paragraph_index' in qa_pairs:
    print("summary_paragraph_index exists!")
    print(f"Values: {list(qa_pairs['summary_paragraph_index'])}")
else:
    print("summary_paragraph_index NOT found in the data")
    
print("\nSample with parent indices:")
for i in range(min(10, len(qa_pairs['question']))):
    parent = qa_pairs['parent_pair_index'][i]
    if 'summary_paragraph_index' in qa_pairs:
        para_idx = qa_pairs['summary_paragraph_index'][i]
        print(f"[{i}] Para: {para_idx}, Parent: {parent}")
    else:
        print(f"[{i}] Parent: {parent}")
    print(f"    Q: {qa_pairs['question'][i][:80]}...")
    print()

Fields in question_summary_pairs:
dict_keys(['question', 'summary', 'parent_pair_index', 'summary_paragraph_index'])

summary_paragraph_index exists!
Values: [np.int32(2), np.int32(2), np.int32(2), np.int32(2), np.int32(3), np.int32(3), np.int32(3), np.int32(3), np.int32(3), np.int32(3), np.int32(4), np.int32(4), np.int32(4), np.int32(0), np.int32(0), np.int32(0), np.int32(0)]

Sample with parent indices:
[0] Para: 2, Parent: -1
    Q: What did NOAA and DOD do following the disbanding of the NPOESS program in 2010?...

[1] Para: 2, Parent: 0
    Q: What specific actions did NOAA take to transfer responsibilities?...

[2] Para: 2, Parent: 0
    Q: What specific actions did DOD take to transfer responsibilities?...

[3] Para: 2, Parent: -1
    Q: How have recent events impacted DOD and NOAA's programs?...

[4] Para: 3, Parent: -1
    Q: How has NOAA developed its satellite system in the past year?...

[5] Para: 3, Parent: 0
    Q: What progress was made in October 2011?...

[6] Para: 3, 

In [4]:
def build_question_hierarchy(qa_pairs):
    """
    Build a hierarchical dictionary representing the question-answer tree.
    
    Returns a dictionary where:
    - Keys are question indices
    - Values contain the question, answer, and list of children
    """
    questions = list(qa_pairs['question'])
    summaries = list(qa_pairs['summary'])
    parent_indices = list(qa_pairs['parent_pair_index'])
    
    # Build the hierarchy dictionary
    hierarchy = {}
    
    # First pass: create all nodes
    for i in range(len(questions)):
        parent_val = parent_indices[i]
        # Convert numpy types to native Python types for JSON serialization
        if isinstance(parent_val, (int, float)):
            parent_val = int(parent_val) if not pd.isna(parent_val) else None
        
        hierarchy[i] = {
            'question': questions[i],
            'summary': summaries[i],
            'parent': parent_val,
            'children': []
        }
    
    # Second pass: link children to parents
    root_nodes = []
    for i in range(len(questions)):
        parent_idx = hierarchy[i]['parent']
        if parent_idx == -1 or parent_idx is None:
            # This is a root node
            root_nodes.append(i)
        else:
            # Add this node as a child of its parent
            if parent_idx in hierarchy:
                hierarchy[parent_idx]['children'].append(i)
    
    return {
        'hierarchy': hierarchy,
        'root_nodes': root_nodes
    }

# Test on first document
test_hierarchy = build_question_hierarchy(train_combined['question_summary_pairs'][0])

print(f"Total questions: {len(test_hierarchy['hierarchy'])}")
print(f"Root questions: {len(test_hierarchy['root_nodes'])}")
print(f"\nRoot node indices: {test_hierarchy['root_nodes']}")

# Show a sample root question and its children
if test_hierarchy['root_nodes']:
    root_idx = test_hierarchy['root_nodes'][0]
    root_node = test_hierarchy['hierarchy'][root_idx]
    print(f"\nSample root question [{root_idx}]:")
    print(f"  Q: {root_node['question'][:150]}...")
    print(f"  Children: {root_node['children']}")
    
    # Show first child if exists
    if root_node['children']:
        child_idx = root_node['children'][0]
        child_node = test_hierarchy['hierarchy'][child_idx]
        print(f"\n  Child question [{child_idx}]:")
        print(f"    Q: {child_node['question'][:150]}...")
        print(f"    Children: {child_node['children']}")

Total questions: 17
Root questions: 7

Root node indices: [0, 3, 4, 6, 10, 13, 14]

Sample root question [0]:
  Q: What did NOAA and DOD do following the disbanding of the NPOESS program in 2010?...
  Children: [1, 2, 5, 11, 12]

  Child question [1]:
    Q: What specific actions did NOAA take to transfer responsibilities?...
    Children: [15]


In [5]:
# Build hierarchies for all documents in train_combined
all_hierarchies = {}

for idx, row in train_combined.iterrows():
    doc_id = row['doc_id']
    qa_pairs = row['question_summary_pairs']
    
    hierarchy = build_question_hierarchy(qa_pairs)
    all_hierarchies[doc_id] = hierarchy

print(f"Built hierarchies for {len(all_hierarchies)} documents")

# Statistics
total_questions = sum(len(h['hierarchy']) for h in all_hierarchies.values())
total_roots = sum(len(h['root_nodes']) for h in all_hierarchies.values())
avg_depth_stats = []

def get_max_depth(hierarchy, node_idx, current_depth=0):
    """Recursively find maximum depth from a node"""
    node = hierarchy[node_idx]
    if not node['children']:
        return current_depth
    return max(get_max_depth(hierarchy, child, current_depth + 1) for child in node['children'])

for doc_id, h in all_hierarchies.items():
    if h['root_nodes']:
        max_depths = [get_max_depth(h['hierarchy'], root) for root in h['root_nodes']]
        avg_depth_stats.append(max(max_depths))

print(f"Total questions across all documents: {total_questions}")
print(f"Total root questions: {total_roots}")
print(f"Average questions per document: {total_questions / len(all_hierarchies):.1f}")
print(f"Average root questions per document: {total_roots / len(all_hierarchies):.1f}")
if avg_depth_stats:
    print(f"Average max depth: {sum(avg_depth_stats) / len(avg_depth_stats):.1f}")
    print(f"Max depth found: {max(avg_depth_stats)}")

# Save to file
import json
with open('data/question_hierarchies.json', 'w') as f:
    json.dump(all_hierarchies, f, indent=2)
    
print("\nSaved hierarchies to data/question_hierarchies.json")

Built hierarchies for 1371 documents
Total questions across all documents: 19878
Total root questions: 9949
Average questions per document: 14.5
Average root questions per document: 7.3
Average max depth: 1.9
Max depth found: 5


TypeError: Object of type int32 is not JSON serializable

In [6]:
def print_tree(hierarchy, node_idx, prefix="", is_last=True, max_q_len=80):
    """Print a tree visualization of the question hierarchy"""
    node = hierarchy[node_idx]
    
    # Print current node
    connector = "└── " if is_last else "├── "
    question_preview = node['question'][:max_q_len] + "..." if len(node['question']) > max_q_len else node['question']
    print(f"{prefix}{connector}[{node_idx}] {question_preview}")
    
    # Print children
    if node['children']:
        extension = "    " if is_last else "│   "
        for i, child_idx in enumerate(node['children']):
            is_last_child = (i == len(node['children']) - 1)
            print_tree(hierarchy, child_idx, prefix + extension, is_last_child, max_q_len)

for doc_i in range(len(all_hierarchies)):
    # Visualize the first document's hierarchy
    first_doc_id = list(all_hierarchies.keys())[doc_i]
    first_hierarchy = all_hierarchies[first_doc_id]

    print(f"Question Hierarchy for document: {first_doc_id}")
    print(f"Total questions: {len(first_hierarchy['hierarchy'])}")
    print(f"Root questions: {len(first_hierarchy['root_nodes'])}\n")

    for i, root_idx in enumerate(first_hierarchy['root_nodes']):
        print_tree(first_hierarchy['hierarchy'], root_idx, "", i == len(first_hierarchy['root_nodes']) - 1)
        if i < len(first_hierarchy['root_nodes']) - 1:
            print()



Question Hierarchy for document: GAO_GAO-12-604
Total questions: 17
Root questions: 7

├── [0] What did NOAA and DOD do following the disbanding of the NPOESS program in 2010?
│   ├── [1] What specific actions did NOAA take to transfer responsibilities?
│   │   └── [15] What issues did the NPOESS program face?
│   ├── [2] What specific actions did DOD take to transfer responsibilities?
│   │   ├── [7] How is the flight project progressing?
│   │   ├── [8] How is the ground project progressing?
│   │   ├── [9] How is the free-flyer program progressing?
│   │   └── [16] How did the government respond to these issues?
│   ├── [5] What progress was made in October 2011?
│   ├── [11] What characterizes the response to these risks?
│   └── [12] What problems might these risks cause?

├── [3] How have recent events impacted DOD and NOAA's programs?

├── [4] How has NOAA developed its satellite system in the past year?

├── [6] What are the three major components of the JPSS program?

├── [10]

In [7]:
def build_question_hierarchy(qa_pairs):
    """
    Build a hierarchical dictionary representing the question-answer tree.
    
    IMPORTANT: parent_pair_index is relative to questions with the same 
    summary_paragraph_index, not global indices!
    
    Returns a dictionary where:
    - Keys are question indices (global)
    - Values contain the question, answer, and list of children
    """
    questions = list(qa_pairs['question'])
    summaries = list(qa_pairs['summary'])
    parent_indices = list(qa_pairs['parent_pair_index'])
    paragraph_indices = list(qa_pairs['summary_paragraph_index'])
    
    # Group questions by paragraph
    paragraph_groups = {}
    for i in range(len(questions)):
        para_idx = int(paragraph_indices[i]) if not pd.isna(paragraph_indices[i]) else 0
        if para_idx not in paragraph_groups:
            paragraph_groups[para_idx] = []
        paragraph_groups[para_idx].append(i)
    
    # Build the hierarchy dictionary
    hierarchy = {}
    
    # First pass: create all nodes with corrected parent indices
    for i in range(len(questions)):
        para_idx = int(paragraph_indices[i]) if not pd.isna(paragraph_indices[i]) else 0
        local_parent_idx = parent_indices[i]
        
        # Convert local parent index to global parent index
        if isinstance(local_parent_idx, (int, float)) and not pd.isna(local_parent_idx):
            local_parent_idx = int(local_parent_idx)
        else:
            local_parent_idx = -1
            
        if local_parent_idx == -1:
            global_parent_idx = None
        else:
            # Map local index within paragraph to global index
            para_questions = paragraph_groups[para_idx]
            if 0 <= local_parent_idx < len(para_questions):
                global_parent_idx = para_questions[local_parent_idx]
            else:
                global_parent_idx = None
        
        hierarchy[i] = {
            'question': questions[i],
            'summary': summaries[i],
            'paragraph': para_idx,
            'parent': global_parent_idx,
            'children': []
        }
    
    # Second pass: link children to parents
    root_nodes = []
    for i in range(len(questions)):
        parent_idx = hierarchy[i]['parent']
        if parent_idx is None:
            # This is a root node
            root_nodes.append(i)
        else:
            # Add this node as a child of its parent
            if parent_idx in hierarchy:
                hierarchy[parent_idx]['children'].append(i)
    
    return {
        'hierarchy': hierarchy,
        'root_nodes': root_nodes,
        'paragraph_groups': paragraph_groups
    }

# Test on first document
test_hierarchy = build_question_hierarchy(train_combined['question_summary_pairs'][0])

print(f"Total questions: {len(test_hierarchy['hierarchy'])}")
print(f"Root questions: {len(test_hierarchy['root_nodes'])}")
print(f"Paragraph groups: {test_hierarchy['paragraph_groups']}")
print(f"\nRoot node indices: {test_hierarchy['root_nodes']}")

# Show a sample with corrected parent
for i in [0, 1, 2, 7, 8]:
    node = test_hierarchy['hierarchy'][i]
    print(f"\n[{i}] Para: {node['paragraph']}, Parent: {node['parent']}")
    print(f"    Q: {node['question'][:100]}...")
    print(f"    Children: {node['children']}")

Total questions: 17
Root questions: 17
Paragraph groups: {2: [0, 1, 2, 3], 3: [4, 5, 6, 7, 8, 9], 4: [10, 11, 12], 0: [13, 14, 15, 16]}

Root node indices: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]

[0] Para: 2, Parent: None
    Q: What did NOAA and DOD do following the disbanding of the NPOESS program in 2010?...
    Children: []

[1] Para: 2, Parent: None
    Q: What specific actions did NOAA take to transfer responsibilities?...
    Children: []

[2] Para: 2, Parent: None
    Q: What specific actions did DOD take to transfer responsibilities?...
    Children: []

[7] Para: 3, Parent: None
    Q: How is the flight project progressing?...
    Children: []

[8] Para: 3, Parent: None
    Q: How is the ground project progressing?...
    Children: []


In [9]:
def build_question_hierarchy(qa_pairs):
    """
    Build a hierarchical dictionary representing the question-answer tree.
    
    IMPORTANT: parent_pair_index is relative to questions with the same 
    summary_paragraph_index, not global indices!
    
    Returns a dictionary where:
    - Keys are question indices (global)
    - Values contain the question, answer, and list of children
    """
    questions = list(qa_pairs['question'])
    summaries = list(qa_pairs['summary'])
    parent_indices = qa_pairs['parent_pair_index']  # Keep as array, don't convert to list yet
    paragraph_indices = qa_pairs['summary_paragraph_index']
    
    # Group questions by paragraph
    paragraph_groups = {}
    for i in range(len(questions)):
        para_idx = int(paragraph_indices[i])
        if para_idx not in paragraph_groups:
            paragraph_groups[para_idx] = []
        paragraph_groups[para_idx].append(i)
    
    # Build the hierarchy dictionary
    hierarchy = {}
    
    # First pass: create all nodes with corrected parent indices
    for i in range(len(questions)):
        para_idx = int(paragraph_indices[i])
        local_parent_idx = int(parent_indices[i])  # Convert numpy int32 to Python int
        
        # Debug for first few items
        if i < 5:
            print(f"DEBUG [{i}]: para={para_idx}, local_parent={local_parent_idx}, type={type(parent_indices[i])}")
            
        if local_parent_idx == -1:
            global_parent_idx = None
        else:
            # Map local index within paragraph to global index
            para_questions = paragraph_groups[para_idx]
            if 0 <= local_parent_idx < len(para_questions):
                global_parent_idx = para_questions[local_parent_idx]
                if i < 5:
                    print(f"  -> Mapping local {local_parent_idx} to global {global_parent_idx}")
            else:
                global_parent_idx = None
                if i < 5:
                    print(f"  -> Out of range!")
        
        hierarchy[i] = {
            'question': questions[i],
            'summary': summaries[i],
            'paragraph': para_idx,
            'parent': global_parent_idx,
            'children': []
        }
    
    # Second pass: link children to parents
    root_nodes = []
    for i in range(len(questions)):
        parent_idx = hierarchy[i]['parent']
        if parent_idx is None:
            # This is a root node
            root_nodes.append(i)
        else:
            # Add this node as a child of its parent
            if parent_idx in hierarchy:
                hierarchy[parent_idx]['children'].append(i)
    
    return {
        'hierarchy': hierarchy,
        'root_nodes': root_nodes,
        'paragraph_groups': paragraph_groups
    }

# Test on first document
print("Testing hierarchy build...")
test_hierarchy = build_question_hierarchy(train_combined['question_summary_pairs'][0])

print(f"\nTotal questions: {len(test_hierarchy['hierarchy'])}")
print(f"Root questions: {len(test_hierarchy['root_nodes'])}")
print(f"Paragraph groups: {test_hierarchy['paragraph_groups']}")
print(f"\nRoot node indices: {test_hierarchy['root_nodes'][:10]}...")

# Show a sample with corrected parent
for i in [0, 1, 2, 7, 8]:
    node = test_hierarchy['hierarchy'][i]
    print(f"\n[{i}] Para: {node['paragraph']}, Parent: {node['parent']}")
    print(f"    Q: {node['question'][:100]}...")
    print(f"    Children: {node['children']}")

Testing hierarchy build...
DEBUG [0]: para=2, local_parent=-1, type=<class 'numpy.int32'>
DEBUG [1]: para=2, local_parent=0, type=<class 'numpy.int32'>
  -> Mapping local 0 to global 0
DEBUG [2]: para=2, local_parent=0, type=<class 'numpy.int32'>
  -> Mapping local 0 to global 0
DEBUG [3]: para=2, local_parent=-1, type=<class 'numpy.int32'>
DEBUG [4]: para=3, local_parent=-1, type=<class 'numpy.int32'>

Total questions: 17
Root questions: 7
Paragraph groups: {2: [0, 1, 2, 3], 3: [4, 5, 6, 7, 8, 9], 4: [10, 11, 12], 0: [13, 14, 15, 16]}

Root node indices: [0, 3, 4, 6, 10, 13, 14]...

[0] Para: 2, Parent: None
    Q: What did NOAA and DOD do following the disbanding of the NPOESS program in 2010?...
    Children: [1, 2]

[1] Para: 2, Parent: 0
    Q: What specific actions did NOAA take to transfer responsibilities?...
    Children: []

[2] Para: 2, Parent: 0
    Q: What specific actions did DOD take to transfer responsibilities?...
    Children: []

[7] Para: 3, Parent: 6
    Q: How is 

In [14]:
# Self-contained cell to build and save question hierarchies
import json
import pandas as pd

def build_question_hierarchy(qa_pairs):
    """Build hierarchical dictionary with paragraph-relative parent indices"""
    questions = list(qa_pairs['question'])
    summaries = list(qa_pairs['summary'])
    parent_indices = qa_pairs['parent_pair_index']
    paragraph_indices = qa_pairs['summary_paragraph_index']
    
    # Group questions by paragraph
    paragraph_groups = {}
    for i in range(len(questions)):
        para_idx = int(paragraph_indices[i])
        if para_idx not in paragraph_groups:
            paragraph_groups[para_idx] = []
        paragraph_groups[para_idx].append(i)
    
    hierarchy = {}
    
    # Create nodes with corrected parent indices
    for i in range(len(questions)):
        para_idx = int(paragraph_indices[i])
        local_parent_idx = int(parent_indices[i])
            
        if local_parent_idx == -1:
            global_parent_idx = None
        else:
            # Map local index within paragraph to global index
            para_questions = paragraph_groups[para_idx]
            if 0 <= local_parent_idx < len(para_questions):
                global_parent_idx = para_questions[local_parent_idx]
            else:
                global_parent_idx = None
        
        hierarchy[i] = {
            'question': questions[i],
            'summary': summaries[i],
            'paragraph': para_idx,
            'parent': global_parent_idx,
            'children': []
        }
    
    # Link children to parents
    root_nodes = []
    for i in range(len(questions)):
        parent_idx = hierarchy[i]['parent']
        if parent_idx is None:
            root_nodes.append(i)
        else:
            if parent_idx in hierarchy:
                hierarchy[parent_idx]['children'].append(i)
    
    return {
        'hierarchy': hierarchy,
        'root_nodes': root_nodes,
        'paragraph_groups': paragraph_groups
    }

def hierarchy_to_nested_dict(hierarchy_data):
    """Convert flat hierarchy to nested dictionary structure"""
    hierarchy = hierarchy_data['hierarchy']
    root_nodes = hierarchy_data['root_nodes']
    
    def build_node(node_idx):
        node = hierarchy[node_idx]
        nested_node = {
            'index': node_idx,
            'question': node['question'],
            'summary': node['summary'],
            'paragraph': node['paragraph'],
            'children': []
        }
        for child_idx in node['children']:
            nested_node['children'].append(build_node(child_idx))
        return nested_node
    
    nested_hierarchy = []
    for root_idx in root_nodes:
        nested_hierarchy.append(build_node(root_idx))
    
    return nested_hierarchy

# Build hierarchies for all documents
all_nested_hierarchies = {}

print("Building nested hierarchies for all documents...")
for idx, row in train_combined.iterrows():
    doc_id = row['doc_id']
    qa_pairs = row['question_summary_pairs']
    
    hierarchy = build_question_hierarchy(qa_pairs)
    nested = hierarchy_to_nested_dict(hierarchy)
    all_nested_hierarchies[doc_id] = nested

print(f"✓ Built hierarchies for {len(all_nested_hierarchies)} documents")

# Save to JSON
with open('data/question_hierarchies_nested.json', 'w') as f:
    json.dump(all_nested_hierarchies, f, indent=2)
    
print("✓ Saved to data/question_hierarchies_nested.json")

Building nested hierarchies for all documents...
✓ Built hierarchies for 1371 documents
✓ Saved to data/question_hierarchies_nested.json


In [15]:
from openai import OpenAI
import pandas as pd

client = OpenAI()

from dotenv import load_dotenv
load_dotenv()

def get_root_questions(qa_pairs):
    """Get only the root (top-level) questions based on hierarchy"""
    parent_indices = qa_pairs['parent_pair_index']
    paragraph_indices = qa_pairs['summary_paragraph_index']
    questions = list(qa_pairs['question'])
    
    # Group questions by paragraph to correctly interpret parent indices
    paragraph_groups = {}
    for i in range(len(questions)):
        para_idx = int(paragraph_indices[i])
        if para_idx not in paragraph_groups:
            paragraph_groups[para_idx] = []
        paragraph_groups[para_idx].append(i)
    
    root_questions = []
    for i in range(len(questions)):
        local_parent_idx = int(parent_indices[i])
        # Root questions have parent = -1
        if local_parent_idx == -1:
            root_questions.append(questions[i])
    
    return root_questions

prompt_base = "Come up with a short phrase that describes the topics covered by the following questions. Questions: " 

for i, row in train_combined.iterrows():
    qa_pairs = row['question_summary_pairs']
    
    # Get only root (top-level) questions
    root_questions = get_root_questions(qa_pairs)
    
    # Build prompt with only root questions
    prompt = prompt_base + "\n".join([f"Q: {q}" for q in root_questions]) + "\n"

    response = client.responses.create(
                model='gpt-5',
                input=prompt
            )
    
    print("Summary phrase: {}".format(response.output_text))
    print("Root questions ({} total): {}".format(len(root_questions), ", ".join(root_questions)))
    print("\n\n\n")

Summary phrase: Transition and status of U.S. polar-orbiting environmental satellites (NPOESS to JPSS)
Root questions (7 total): What did NOAA and DOD do following the disbanding of the NPOESS program in 2010?, How have recent events impacted DOD and NOAA's programs?, How has NOAA developed its satellite system in the past year?, What are the three major components of the JPSS program?, What risks does the JPSS program face?, Why are environmental satellites important?, What is the NPOESS?




Summary phrase: Federal performance measurement and reporting—APGs, CAP goals, and GPRA Modernization Act compliance
Root questions (9 total): What did GAO's review of APGs for 2012 to 2013 reveal?, What did agencies fail to fully explain?, What does the act direct agencies to identify?, What does the act require regarding CAP goals?, What characterized the performance measures of APGs?, What did agencies fail to identify?, What has GAO's work repeatedly shown regarding federal agencies?, What re

KeyboardInterrupt: 

In [16]:
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import pandas as pd

client = OpenAI()

from dotenv import load_dotenv
load_dotenv()

def get_root_questions(qa_pairs):
    """Get only the root (top-level) questions based on hierarchy"""
    parent_indices = qa_pairs['parent_pair_index']
    paragraph_indices = qa_pairs['summary_paragraph_index']
    questions = list(qa_pairs['question'])
    
    # Group questions by paragraph to correctly interpret parent indices
    paragraph_groups = {}
    for i in range(len(questions)):
        para_idx = int(paragraph_indices[i])
        if para_idx not in paragraph_groups:
            paragraph_groups[para_idx] = []
        paragraph_groups[para_idx].append(i)
    
    root_questions = []
    for i in range(len(questions)):
        local_parent_idx = int(parent_indices[i])
        # Root questions have parent = -1
        if local_parent_idx == -1:
            root_questions.append(questions[i])
    
    return root_questions

def process_row(args):
    """Process a single row to generate summary phrase from root questions only"""
    idx, row = args
    qa_pairs = row['question_summary_pairs']
    doc_id = row['doc_id']
    id_val = row['id']
    
    # Get only root (top-level) questions
    root_questions = get_root_questions(qa_pairs)
    
    # Build prompt with only root questions
    prompt = "Come up with a short phrase that describes the topics covered by the following questions. Questions: \n"
    prompt += "\n".join([f"Q: {q}" for q in root_questions]) + "\n"
    
    try:
        response = client.responses.create(
            model='gpt-5',
            input=prompt
        )
        return {
            'index': idx,
            'summary_phrase': response.output_text,
            'root_questions': root_questions,
            'num_root_questions': len(root_questions),
            'success': True, 
            'doc_id': doc_id,
            'id': id_val
        }
    except Exception as e:
        return {
            'index': idx,
            'error': str(e),
            'success': False,
            'doc_id': doc_id,
            'id': id_val
        }

# Process with thread pool
results = []
max_workers = 10  # Adjust based on your rate limits

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    # Submit all jobs
    futures = {
        executor.submit(process_row, (i, row)): i 
        for i, row in train_combined.iterrows()
    }
    
    # Process completed jobs with progress bar
    for future in tqdm(as_completed(futures), total=len(futures), desc="Generating phrases"):
        result = future.result()
        results.append(result)
        
        if result['success']:
            print(f"\nRow {result['index']} ({result['doc_id']}) - {result['num_root_questions']} root questions")
            print(f"  Phrase: {result['summary_phrase']}")

# Save results
results_df = pd.DataFrame(results)
results_df.to_csv('batch_results_root_only.csv', index=False)

print(f"\n✓ Completed {len(results)} documents")
print(f"✓ Saved to batch_results_root_only.csv")

# Show statistics
successful = results_df[results_df['success'] == True]
print(f"\nSuccess rate: {len(successful)}/{len(results_df)}")
print(f"Average root questions per document: {successful['num_root_questions'].mean():.1f}")

Generating phrases:   0%|          | 1/1371 [00:04<1:35:08,  4.17s/it]


Row 7 (GAO_GAO-17-25) - 5 root questions
  Phrase: GAO review of the Architect of the Capitol OIG—its purpose, audit planning limits, FY2014 changes, report scope, and recommendations.


Generating phrases:   0%|          | 3/1371 [00:05<28:31,  1.25s/it]  


Row 6 (GAO_GAO-12-610) - 4 root questions
  Phrase: GAO inquiry into HRSA grant compliance and CARE Act HIV/AIDS patient support

Row 9 (GAO_GAO-13-687) - 4 root questions
  Phrase: Governance of bi-state tolling authorities: powers, transparency, authorization.


Generating phrases:   0%|          | 4/1371 [00:05<23:04,  1.01s/it]

...


Row 1367 (CRS_RL33344) - 7 root questions
  Phrase: FY2007 federal appropriations—HUD funding, funding levels, CRs, and Senate passage

Row 1366 (CRS_R43221) - 6 root questions
  Phrase: Immigrant public benefits: PRWORA, constitutional debates, and state actions


Generating phrases: 100%|█████████▉| 1370/1371 [15:09<00:01,  1.51s/it]


Row 1368 (CRS_R44845) - 6 root questions
  Phrase: Federal Direct Loan lifecycle and FSA administration—from application to default.


Generating phrases: 100%|██████████| 1371/1371 [15:24<00:00,  1.48it/s]


Row 1326 (GAO_GAO-18-72) - 4 root questions
  Phrase: GAO assessment of four agencies’ methodologies, shared challenges, and public vs. sensitive reporting differences

✓ Completed 1371 documents
✓ Saved to batch_results_root_only.csv

Success rate: 1371/1371
Average root questions per document: 7.3


In [17]:
import pandas as pd 

# Load the batch results with summary phrases
phrases_df = pd.read_csv('batch_results_root_only.csv')

# IMPORTANT: Sort by index to align with train_combined
# as_completed() returns results in completion order, not submission order
phrases_df = phrases_df.sort_values('index').reset_index(drop=True)

# Merge the summary phrases back into train_combined
train_combined = train_combined.reset_index(drop=True)
train_combined['summary_phrase'] = phrases_df['summary_phrase']

# Create the summarization prompt column
train_combined['summarization_prompt'] = train_combined['summary_phrase'].apply(
    lambda phrase: f"Summarize the following document, focusing on {phrase}"
)

# Verify it worked - check alignment
print(f"Added 'summarization_prompt' column to train_combined")
print(f"\nExample (row 0):")
print(f"Doc ID: {train_combined['doc_id'].iloc[0]}")
print(f"Summary phrase: {train_combined['summary_phrase'].iloc[0]}")
print(f"First question: {list(train_combined['question_summary_pairs'].iloc[0]['question'])[0]}")
print(f"\nColumns: {train_combined.columns.tolist()}")

Added 'summarization_prompt' column to train_combined

Example (row 0):
Doc ID: GAO_GAO-12-604
Summary phrase: NOAA/DOD environmental satellites: NPOESS aftermath, JPSS components and risks
First question: What did NOAA and DOD do following the disbanding of the NPOESS program in 2010?

Columns: ['doc_id', 'document_sections', 'question_summary_pairs', 'id', 'document', 'summary', 'summary_phrase', 'summarization_prompt']


In [ ]:
from together import Together 
import dotenv
import pandas as pd
import tqdm 
dotenv.load_dotenv()

train_df = pd.read_csv('data/train_combined_with_prompts.csv')
print(train_df.columns.tolist())
from openai import OpenAI
results = {}
# Generate summaries for all documents 
for index, row in tqdm.tqdm(train_df.iterrows(), total=train_df.shape[0]):
    document = row['document']
    topic_prompt = row['summarization_prompt']
    base_prompt = "Summarize the following document: " 
    overall_topic_prompt = "{} \n\n Document: {}".format(topic_prompt, document)
    overall_base_prompt = "{} \n\n Document: {}".format(base_prompt, document)
    client = Together() 

    thinking_base_response = client.chat.completions.create( 
        model="Qwen/Qwen3-235B-A22B-fp8-tput",
        messages=[ 
            {
                "role": "user", 
                "content": overall_base_prompt + "/think",
                "enable_thinking": True
            } 
        ] 
    )  
    thinking_base_summary = thinking_base_response.choices[0].message.content

    thinking_topic_response = client.chat.completions.create( 
        model="Qwen/Qwen3-235B-A22B-fp8-tput",
        messages=[ 
            {
                "role": "user", 
                "content": overall_topic_prompt + "/think",
                "enable_thinking": True
            } 
        ] 
    )  
    thinking_topic_summary = thinking_topic_response.choices[0].message.content

    non_thinking_base_response = client.chat.completions.create( 
        model="Qwen/Qwen3-235B-A22B-fp8-tput",
        messages=[ 
            {
                "role": "user", 
                "content": overall_base_prompt + "/no_think",
                "enable_thinking": False
            } 
        ] 
    )  
    non_thinking_base_summary = non_thinking_base_response.choices[0].message.content

    non_thinking_topic_response = client.chat.completions.create( 
        model="Qwen/Qwen3-235B-A22B-fp8-tput",
        messages=[ 
            {
                "role": "user", 
                "content": overall_topic_prompt + "/no_think",
                "enable_thinking": False
            } 
        ] 
    )  
    non_thinking_topic_summary = non_thinking_topic_response.choices[0].message.content
    results[row['id']] = {'thinking_base_summary': thinking_base_summary, 'thinking_topic_summary': thinking_topic_summary, 'non_thinking_base_summary': non_thinking_base_summary, 'non_thinking_topic_summary': non_thinking_topic_summary}
    
results_df = pd.DataFrame.from_dict(results, orient='index', columns=['thinking_base_summary', 'thinking_topic_summary', 'non_thinking_base_summary', 'non_thinking_topic_summary'])
results_df.to_csv('data/regular_generated_summaries.csv')

['doc_id', 'document_sections', 'question_summary_pairs', 'id', 'document', 'summary', 'summary_phrase', 'summarization_prompt']


KeyboardInterrupt: 